In [1]:
# =======================================================================
# CELL 0: ONE-SHOT SETUP (REQUIRED FOR PROTOBUF ON KAGGLE)
# =======================================================================

import subprocess
import sys

# Step 1: Fix protobuf
print("[1/3] Removing old protobuf...")
subprocess.run([sys.executable, "-m", "pip", "uninstall", "-y", "protobuf"], 
 capture_output=True, check=False)
subprocess.run([sys.executable, "-m", "pip", "uninstall", "-y", "protobuf"], 
 capture_output=True, check=False)
print(" ✓ Old protobuf removed")

# Step 2: Install packages
print("[2/3] Installing protobuf 3.20.3...")
subprocess.run([sys.executable, "-m", "pip", "install", "-q", "protobuf==3.20.3"], 
 capture_output=True, check=True)
print(" ✓ Protobuf 3.20.3 installed")

print("[3/3] Installing packages...")
subprocess.run([
 sys.executable, "-m", "pip", "install", "-q",
 "pandas", "numpy", "scikit-learn",
 "transformers", "torch", "accelerate", "peft",
 "datasets", "sentencepiece", "evaluate", "rouge_score",
 "sentence-transformers", "sacrebleu", "bert_score"
], capture_output=True, check=True)
print(" ✓ Packages installed")


[1/3] Removing old protobuf...
 ✓ Old protobuf removed
[2/3] Installing protobuf 3.20.3...
 ✓ Protobuf 3.20.3 installed
[3/3] Installing packages...
 ✓ Packages installed


In [2]:
# =======================================================================
# CELL 1: IMPORTS & GPU CHECK
# =======================================================================

import os, gc, csv, json, re, time, random
import pandas as pd, numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from pathlib import Path
from collections import Counter, defaultdict
from datasets import Dataset
from transformers import (
 T5TokenizerFast,
 T5ForConditionalGeneration,
 Seq2SeqTrainingArguments,
 Seq2SeqTrainer,
 DataCollatorForSeq2Seq,
 GenerationConfig
)
from peft import PeftModel
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings('ignore')

# Environment optimization
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

print("✅ All libraries imported successfully")

# GPU check
if torch.cuda.is_available():
 DEVICE = torch.device("cuda")
 print(f"✅ GPU: {torch.cuda.get_device_name(0)}")
 print(f" Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
 DEVICE = torch.device("cpu")
 print("⚠️ No GPU - using CPU")


2026-01-26 21:03:32.429906: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1769461412.689913      24 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1769461412.761939      24 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1769461413.370523      24 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769461413.370570      24 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769461413.370572      24 computation_placer.cc:177] computation placer alr

✅ All libraries imported successfully
✅ GPU: Tesla T4
 Memory: 15.8 GB


In [3]:
# =======================================================================
# CELL 2: CONFIGURATION
# =======================================================================

# --- PATHS ---
TSV_DATA_DIR = Path("/kaggle/input/new-absa-baseline-few-shots")  # Few-shot examples
assert TSV_DATA_DIR.exists(), f"TSV_DATA_DIR missing: {TSV_DATA_DIR}"

# DAPT adapter from span-masked 700k training
DAPT_ADAPTER_DIR = Path("/kaggle/input/t5-base-700k-masked-span/flan-t5-base-social-lora-masked-span")
assert DAPT_ADAPTER_DIR.exists(), f"DAPT_ADAPTER_DIR missing: {DAPT_ADAPTER_DIR}"

# Output directory
OUTPUT_MODEL_DIR = Path("/kaggle/working/soft-cl-fewshot-absa")
OUTPUT_MODEL_DIR.mkdir(parents=True, exist_ok=True)

BASE_MODEL_NAME = "google/flan-t5-base"

# --- HYPERPARAMETERS ---
MAX_INPUT_LENGTH = 512  # Few-shot prompts are longer
MAX_TARGET_LENGTH = 128  # Allow longer targets for multi-aspect

# Batch sizes (conservative for T4 with dual models)
PER_DEVICE_TRAIN_BATCH_SIZE = 4
GRADIENT_ACCUMULATION_STEPS = 8  # Effective batch = 32
PER_DEVICE_EVAL_BATCH_SIZE = 4

# Learning rate
LEARNING_RATE = 2e-4
WARMUP_RATIO = 0.05
WEIGHT_DECAY = 0.01

# Training duration
NUM_TRAIN_EPOCHS = 5

# Soft CL hyperparameter
ALPHA_CL = 0.1  # Weight for contrastive loss

# Generation settings
GEN_NUM_BEAMS = 5

# Logging
LOGGING_STEPS = 25


In [4]:
# =======================================================================
# CELL 3: LOAD AND VERIFY TSV DATA WITH FEW-SHOT EXAMPLES
# =======================================================================

def load_tsv_simple(path):
    """Load TSV with input/target columns, skip header."""
    rows = []
    print(f"Reading from: {path.name}...")
    
    with open(path, "r", encoding="utf-8") as fh:
        reader = csv.reader(fh, delimiter='\t', quoting=csv.QUOTE_MINIMAL)
        header = next(reader, None)
        print(f" Header: {header}")
        
        for row in reader:
            if len(row) >= 2:
                rows.append({"input": row[0], "target": row[1]})
    
    print(f" ✓ Loaded {len(rows)} rows")
    return rows

# Load datasets
train_rows = load_tsv_simple(TSV_DATA_DIR / "train.tsv")
val_rows = load_tsv_simple(TSV_DATA_DIR / "val.tsv")
test_rows = load_tsv_simple(TSV_DATA_DIR / "test.tsv")

# Convert to HuggingFace Datasets
train_ds = Dataset.from_pandas(pd.DataFrame(train_rows))
val_ds = Dataset.from_pandas(pd.DataFrame(val_rows))
test_ds = Dataset.from_pandas(pd.DataFrame(test_rows))

print(f"Dataset sizes:")
print(f" Train: {len(train_ds):,}")
print(f" Val: {len(val_ds):,}")
print(f" Test: {len(test_ds):,}")

# VERIFICATION: Print a FULL random entry
random_idx = random.randint(0, len(train_ds) - 1)
sample = train_ds[random_idx]

print(f"\n[Index: {random_idx}]")
print("="*70)
print("FULL INPUT:")
print("="*70)
print(sample['input'])

print("\n" + "="*70)
print("FULL TARGET:")
print("="*70)
print(sample['target'])

# Parse and verify target format (7-field: aspect|sentiment|span|rationale|is_implicit|has_slang|has_emoji)
target = sample['target']
entries = target.split(' ; ')
print(f"\nNumber of aspects: {len(entries)}")

for i, entry in enumerate(entries, 1):
    fields = [f.strip() for f in entry.split(' | ')]
    print(f"\n Entry {i}: {len(fields)} fields")
    if len(fields) == 7:
        print(f" ✅ Correct 7-field format")
        print(f" [0] aspect: {fields[0]}")
        print(f" [1] sentiment: {fields[1]}")
        print(f" [2] span: {fields[2][:60]}...")
        print(f" [3] rationale: {fields[3][:60]}...")
        print(f" [4] is_implicit: {fields[4]}")
        print(f" [5] has_slang: {fields[5]}")
        print(f" [6] has_emoji: {fields[6]}")


Reading from: train.tsv...
 Header: ['input_text', 'target_text']
 ✓ Loaded 2114 rows
Reading from: val.tsv...
 Header: ['input_text', 'target_text']
 ✓ Loaded 452 rows
Reading from: test.tsv...
 Header: ['input_text', 'target_text']
 ✓ Loaded 453 rows
Dataset sizes:
 Train: 2,114
 Val: 452
 Test: 453

[Index: 401]
FULL INPUT:
Extract aspects, sentiments, and rationales. Output format: aspect | sentiment | span | rationale | is_implicit | has_slang | has_emoji
Separate multiple aspects with ' ; '.

Example 1 (Explicit, Multi-aspect):
Text: Great food, slow service.
Output:
food | positive | Great food | User explicitly praises food quality. | FALSE | FALSE | FALSE ; service | negative | slow service | User explicitly complains about service speed. | FALSE | FALSE | FALSE

Example 2 (Slang + Emoji):
Text: ugh battery draining so fast 😩
Output:
battery_life | negative | battery draining so fast | Explicit complaint about rapid battery drain with frustration emoji. | FALSE | TRUE | TRUE



In [5]:
# =======================================================================
# CELL 4: LOAD TOKENIZER AND MODELS
# =======================================================================

gc.collect()
torch.cuda.empty_cache()

# --- 1. Load Tokenizer ---
print("\n[1/4] Loading tokenizer...")
tokenizer = T5TokenizerFast.from_pretrained(BASE_MODEL_NAME)
print(f" ✓ Tokenizer loaded: vocab size = {tokenizer.vocab_size}")

# --- 2. Load Base Model ---
print("\n[2/4] Loading base model...")
base_model = T5ForConditionalGeneration.from_pretrained(BASE_MODEL_NAME)
print(f" ✓ Base model loaded: {sum(p.numel() for p in base_model.parameters()):,} parameters")

# --- 3. Load DAPT Adapter and Merge ---
print("\n[3/4] Loading and merging DAPT adapter...")
print(f" Loading from: {DAPT_ADAPTER_DIR}")
model_with_dapt = PeftModel.from_pretrained(base_model, str(DAPT_ADAPTER_DIR))
print(" ✓ DAPT adapter attached")

print(" Merging adapter weights...")
model = model_with_dapt.merge_and_unload()
print(" ✓ Adapter merged successfully!")

# Unfreeze all parameters for full fine-tuning
for param in model.parameters():
 param.requires_grad = True

model = model.to(DEVICE)

# --- 4. Load Teacher Model (Frozen) ---
print("\n[4/4] Loading Teacher model (frozen, on CPU)...")
teacher = SentenceTransformer('all-mpnet-base-v2')
teacher = teacher.to('cpu')
teacher.eval()
for param in teacher.parameters():
 param.requires_grad = False
print(f" ✓ Teacher model loaded on CPU")

# Verify trainable params
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print("\n" + "="*70)
print("MODELS READY")
print("="*70)
print(f"Student (T5+DAPT):")
print(f" Total params: {total_params:,}")
print(f" Trainable params: {trainable_params:,}")
print(f" Device: {DEVICE}")
print(f"Teacher (MPNet):")
print(f" Device: CPU (frozen)")
print("="*70)



[1/4] Loading tokenizer...


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

 ✓ Teacher model loaded on CPU

MODELS READY
Student (T5+DAPT):
 Total params: 247,577,856
 Trainable params: 247,577,856
 Device: cuda
Teacher (MPNet):
 Device: CPU (frozen)


In [6]:
# =======================================================================
# CELL 5: TOKENIZING DATASETS
# =======================================================================

def preprocess_function(batch):
    """Tokenize input/target pairs."""
    inputs = tokenizer(
        batch["input"],
        max_length=MAX_INPUT_LENGTH,
        truncation=True,
        padding=False
    )
    
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            batch["target"],
            max_length=MAX_TARGET_LENGTH,
            truncation=True,
            padding=False
        )
    
    inputs["labels"] = labels["input_ids"]
    return inputs

print("\nTokenizing train set...")
tokenized_train = train_ds.map(
    preprocess_function,
    batched=True,
    remove_columns=train_ds.column_names,
    desc="Tokenizing train"
)

print("Tokenizing validation set...")
tokenized_val = val_ds.map(
    preprocess_function,
    batched=True,
    remove_columns=val_ds.column_names,
    desc="Tokenizing val"
)

print("Tokenizing test set...")
tokenized_test = test_ds.map(
    preprocess_function,
    batched=True,
    remove_columns=test_ds.column_names,
    desc="Tokenizing test"
)

# Statistics
train_input_lens = [len(x) for x in tokenized_train['input_ids']]
train_label_lens = [len(x) for x in tokenized_train['labels']]

print("\n" + "="*70)
print("TOKENIZATION STATISTICS")
print("="*70)
print(f"Input tokens - Min: {min(train_input_lens)}, Max: {max(train_input_lens)}, Avg: {np.mean(train_input_lens):.1f}")
print(f"Target tokens - Min: {min(train_label_lens)}, Max: {max(train_label_lens)}, Avg: {np.mean(train_label_lens):.1f}")

truncated_inputs = sum(1 for l in train_input_lens if l == MAX_INPUT_LENGTH)
truncated_targets = sum(1 for l in train_label_lens if l == MAX_TARGET_LENGTH)

print(f"\nTruncation:")
print(f" Inputs: {truncated_inputs}/{len(train_input_lens)} ({truncated_inputs/len(train_input_lens)*100:.1f}%)")
print(f" Targets: {truncated_targets}/{len(train_label_lens)} ({truncated_targets/len(train_label_lens)*100:.1f}%)")



Tokenizing train set...


Tokenizing train:   0%|          | 0/2114 [00:00<?, ? examples/s]

Tokenizing validation set...


Tokenizing val:   0%|          | 0/452 [00:00<?, ? examples/s]

Tokenizing test set...


Tokenizing test:   0%|          | 0/453 [00:00<?, ? examples/s]


TOKENIZATION STATISTICS
Input tokens - Min: 512, Max: 512, Avg: 512.0
Target tokens - Min: 24, Max: 128, Avg: 68.7

Truncation:
 Inputs: 2114/2114 (100.0%)
 Targets: 183/2114 (8.7%)


In [7]:
# # =======================================================================
# # CELL 6: SOFT CONTRASTIVE LEARNING TRAINER (FIXED)
# # =======================================================================

# print("="*70)
# print("CELL 6: DEFINING SOFT CL TRAINER")
# print("="*70)

# class SoftCLTrainer(Seq2SeqTrainer):
#     """
#     Custom Trainer implementing Soft Contrastive Learning:
    
#     Loss = L_generative + alpha * L_soft_cl
    
#     Where:
#     - L_generative: Standard cross-entropy for seq2seq generation
#     - L_soft_cl: MSE between student and teacher similarity matrices
    
#     ✅ FIXED: Teacher now encodes ONLY field [3] (rationale)
#     ✅ FIXED: field[3].strip() applied immediately before appending
#     """
    
#     def __init__(self, teacher_model, tokenizer_ref, alpha=0.1, *args, **kwargs):
#         super().__init__(*args, **kwargs)
#         self.teacher = teacher_model
#         self.tokenizer_ref = tokenizer_ref
#         self.alpha = alpha
#         self.eps = 1e-8
    
#     def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
#         # =====================================================================
#         # A. GENERATIVE LOSS
#         # =====================================================================
#         outputs = model(**inputs)
#         gen_loss = outputs.loss
        
#         if gen_loss.dim() > 0:
#             gen_loss = gen_loss.mean()
        
#         # =====================================================================
#         # B. SOFT CONTRASTIVE LOSS
#         # =====================================================================
#         base_model = getattr(model, "module", model)
        
#         # B1. Get student encoder hidden states
#         encoder_outputs = base_model.encoder(
#             input_ids=inputs['input_ids'],
#             attention_mask=inputs['attention_mask']
#         )
        
#         hidden_states = encoder_outputs.last_hidden_state
#         attention_mask = inputs['attention_mask'].unsqueeze(-1)
        
#         # Masked mean pooling
#         masked_hidden = hidden_states * attention_mask
#         sum_hidden = masked_hidden.sum(dim=1)
#         sum_mask = attention_mask.sum(dim=1).clamp(min=self.eps)
#         student_embeds = sum_hidden / sum_mask
        
#         # L2 normalize
#         student_norm = student_embeds / (student_embeds.norm(dim=1, keepdim=True) + self.eps)
        
#         # B2. Get teacher embeddings for TARGET RATIONALES ONLY ✅ FIXED
#         with torch.no_grad():
#             label_ids = inputs['labels'].clone()
#             label_ids[label_ids == -100] = self.tokenizer_ref.pad_token_id
#             target_texts = self.tokenizer_ref.batch_decode(label_ids, skip_special_tokens=True)
            
#             # ✅ EXTRACT ONLY FIELD [3] (RATIONALE)
#             # Target format: aspect | sentiment | span | RATIONALE | is_implicit | has_slang | has_emoji
#             rationales = []
#             for target_text in target_texts:
#                 # Handle multiple aspects separated by ' ; '
#                 entries = target_text.split(' ; ')
#                 aspect_rationales = []
                
#                 for entry in entries:
#                     fields = [f.strip() for f in entry.split(' | ')]
                    
#                     # Extract field [3] which is the rationale
#                     if len(fields) >= 4:
#                         rationale = fields[3].strip()  # ✅ FIXED: Strip immediately
#                         if rationale:  # Skip empty rationales
#                             aspect_rationales.append(rationale)
                
#                 # Combine multiple aspects' rationales into single text
#                 combined_rationale = " ".join(aspect_rationales)
#                 rationales.append(combined_rationale)
            
#             # ✅ NOW encode only the extracted rationales
#             teacher_embeds = self.teacher.encode(
#                 rationales,  # Only rationales, not full 7-field output!
#                 convert_to_tensor=True,
#                 show_progress_bar=False
#             )
            
#             teacher_norm = teacher_embeds / (teacher_embeds.norm(dim=1, keepdim=True) + self.eps)
        
#         # B3. Compute similarity matrices (on CPU for stability)
#         student_cpu = student_norm.cpu()
#         student_sim = torch.mm(student_cpu, student_cpu.t())
#         teacher_sim = torch.mm(teacher_norm, teacher_norm.t())
        
#         # B4. MSE loss between similarity matrices
#         cl_loss = F.mse_loss(student_sim, teacher_sim)
        
#         # =====================================================================
#         # C. COMBINED LOSS
#         # =====================================================================
#         total_loss = gen_loss + self.alpha * cl_loss.to(gen_loss.device)
        
#         # Logging
#         if self.state.global_step % LOGGING_STEPS == 0 and self.state.global_step > 0:
#             print(f" [Step {self.state.global_step:4d}] Gen: {gen_loss.item():.4f} | CL: {cl_loss.item():.4f} | Total: {total_loss.item():.4f}")
        
#         return (total_loss, outputs) if return_outputs else total_loss

# print("✅ SoftCLTrainer class defined (FIXED)")
# print(f" Alpha (CL weight): {ALPHA_CL}")
# print("\n✅ CELL 6 COMPLETE")


In [8]:
# # =======================================================================
# # CELL 7: TRAINING SETUP
# # =======================================================================

# data_collator = DataCollatorForSeq2Seq(
#     tokenizer=tokenizer,
#     model=model,
#     padding=True
# )

# training_args = Seq2SeqTrainingArguments(
#     output_dir=str(OUTPUT_MODEL_DIR),
    
#     per_device_train_batch_size=PER_DEVICE_TRAIN_BATCH_SIZE,
#     per_device_eval_batch_size=PER_DEVICE_EVAL_BATCH_SIZE,
#     gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
    
#     learning_rate=LEARNING_RATE,
#     warmup_ratio=WARMUP_RATIO,
#     weight_decay=WEIGHT_DECAY,
#     optim="adamw_torch",
    
#     num_train_epochs=NUM_TRAIN_EPOCHS,
    
#     eval_strategy="epoch",
#     save_strategy="epoch",
#     load_best_model_at_end=True,
#     metric_for_best_model="eval_loss",
#     greater_is_better=False,
#     save_total_limit=2,
    
#     predict_with_generate=True,
#     generation_max_length=MAX_TARGET_LENGTH,
#     generation_num_beams=GEN_NUM_BEAMS,
#     generation_config=GenerationConfig(use_cache=False),
    
#     fp16=True,
#     gradient_checkpointing=True,
#     max_grad_norm=1.0,
    
#     logging_strategy="steps",
#     logging_steps=LOGGING_STEPS,
#     report_to="none",
    
#     dataloader_num_workers=0,
#     dataloader_pin_memory=False,
# )

# trainer = SoftCLTrainer(
#     teacher_model=teacher,
#     tokenizer_ref=tokenizer,
#     alpha=ALPHA_CL,
#     model=model,
#     args=training_args,
#     train_dataset=tokenized_train,
#     eval_dataset=tokenized_val,
#     tokenizer=tokenizer,
#     data_collator=data_collator,
# )

# steps_per_epoch = len(tokenized_train) // (PER_DEVICE_TRAIN_BATCH_SIZE * GRADIENT_ACCUMULATION_STEPS)
# total_steps = steps_per_epoch * NUM_TRAIN_EPOCHS

# print("\n" + "="*70)
# print("TRAINING CONFIGURATION")
# print("="*70)
# print(f"Effective batch size: {PER_DEVICE_TRAIN_BATCH_SIZE * GRADIENT_ACCUMULATION_STEPS}")
# print(f"Training samples: {len(tokenized_train)}")
# print(f"Steps per epoch: {steps_per_epoch}")
# print(f"Total training steps: ~{total_steps}")
# print("="*70)


In [9]:
# # =======================================================================
# # CELL 8: STARTING SOFT CL TRAINING
# # =======================================================================

# print("="*70)
# print("CELL 8: STARTING SOFT CL TRAINING")
# print("="*70)
# print(f"Model: FLAN-T5-Base + DAPT (Span-Masked 700k)")
# print(f"Task: ABSA with Soft Contrastive Learning")
# print(f"Dataset: {len(tokenized_train)} train, {len(tokenized_val)} val")
# print(f"Alpha (CL Weight): {ALPHA_CL}")
# print("="*70)

# start_time = time.time()
# model.train()

# train_result = trainer.train()

# end_time = time.time()
# training_minutes = (end_time - start_time) / 60

# print("\n" + "="*70)
# print("TRAINING COMPLETE")
# print("="*70)
# print(f"Training time: {training_minutes:.1f} minutes ({training_minutes/60:.2f} hours)")
# print(f"Final train loss: {train_result.metrics.get('train_loss', 'N/A'):.4f}")
# print("="*70)

# trainer.log_metrics("train", train_result.metrics)
# trainer.save_metrics("train", train_result.metrics)

# print(f"\nSaving best model to {OUTPUT_MODEL_DIR}...")
# trainer.save_model(str(OUTPUT_MODEL_DIR))
# tokenizer.save_pretrained(str(OUTPUT_MODEL_DIR))
# print("✅ Model and tokenizer saved")


In [10]:
# =======================================================================
# CELL 7.5: PREPARE DATA COLLATOR (ADD THIS NEW CELL)
# =======================================================================

from transformers import DataCollatorForSeq2Seq

print("="*70)
print("CELL 7.5: DATA COLLATOR SETUP")
print("="*70)

def preprocess_function(examples):
    """
    Tokenize the input and target texts.
    Handles both single examples and batches.
    """
    # Handle single example or batch
    if isinstance(examples['input'], str):
        inputs = [examples['input']]
        targets = [examples['target']]
    else:
        inputs = examples['input']
        targets = examples['target']
    
    # Tokenize inputs
    model_inputs = tokenizer(
        inputs,
        max_length=512,
        padding=False,  # Padding done by data collator
        truncation=True,
        return_tensors=None
    )
    
    # Tokenize targets
    labels = tokenizer(
        targets,
        max_length=256,
        padding=False,
        truncation=True,
        return_tensors=None
    )
    
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Apply tokenization to datasets
print("\n📦 Tokenizing datasets...")
train_dataset_tokenized = train_ds.map(
    preprocess_function,
    batched=True,
    remove_columns=train_ds.column_names,  # Remove original columns
    desc="Tokenizing train dataset"
)

val_dataset_tokenized = val_ds.map(
    preprocess_function,
    batched=True,
    remove_columns=val_ds.column_names,
    desc="Tokenizing validation dataset"
)

test_dataset_tokenized = test_ds.map(
    preprocess_function,
    batched=True,
    remove_columns=test_ds.column_names,
    desc="Tokenizing test dataset"
)

print(f"\n✅ Tokenized datasets:")
print(f"  Train: {len(train_dataset_tokenized)} samples")
print(f"  Val: {len(val_dataset_tokenized)} samples")
print(f"  Test: {len(test_dataset_tokenized)} samples")
print(f"  Columns: {train_dataset_tokenized.column_names}")

# Create data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    padding=True,
    return_tensors="pt"
)

print("\n✅ CELL 7.5 COMPLETE - Data ready for training")


CELL 7.5: DATA COLLATOR SETUP

📦 Tokenizing datasets...


Tokenizing train dataset:   0%|          | 0/2114 [00:00<?, ? examples/s]

Tokenizing validation dataset:   0%|          | 0/452 [00:00<?, ? examples/s]

Tokenizing test dataset:   0%|          | 0/453 [00:00<?, ? examples/s]


✅ Tokenized datasets:
  Train: 2114 samples
  Val: 452 samples
  Test: 453 samples
  Columns: ['input_ids', 'attention_mask', 'labels']

✅ CELL 7.5 COMPLETE - Data ready for training


In [11]:
# =======================================================================
# CELL 8: TRAINING (FIXED VERSION v5)
# =======================================================================

print("="*70)
print("CELL 8: MODEL TRAINING (FIXED)")
print("="*70)

from transformers import TrainingArguments, Trainer, EarlyStoppingCallback
import torch.nn.functional as F

# =====================================================================
# CRITICAL FIX: Reduce CL weight
# =====================================================================
CL_WEIGHT = 0.05  # ✅ Much smaller than generation loss

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        """Custom loss with REDUCED contrastive learning."""
        outputs = model(**inputs)
        
        # Generation loss (main objective)
        gen_loss = outputs.loss
        if gen_loss.dim() > 0:  # ✅ Handle multi-element tensor
            gen_loss = gen_loss.mean()
        
        # Contrastive loss (auxiliary)
        if hasattr(outputs, 'encoder_last_hidden_state'):
            encoder_hidden = outputs.encoder_last_hidden_state
            batch_size = encoder_hidden.size(0)
            
            if batch_size > 1:
                # Compute similarities
                encoder_hidden_norm = F.normalize(encoder_hidden.mean(dim=1), p=2, dim=1)
                similarity_matrix = torch.matmul(encoder_hidden_norm, encoder_hidden_norm.t())
                
                # Contrastive loss with temperature
                labels = torch.arange(batch_size).to(similarity_matrix.device)
                cl_loss = F.cross_entropy(similarity_matrix / 0.5, labels)
                
                # ✅ CRITICAL: Much smaller weight
                total_loss = gen_loss + CL_WEIGHT * cl_loss
                
                # Log individual losses (safely)
                if self.state.global_step % 25 == 0:
                    gen_val = gen_loss.item() if isinstance(gen_loss, torch.Tensor) else gen_loss
                    cl_val = cl_loss.item() if isinstance(cl_loss, torch.Tensor) else cl_loss
                    total_val = total_loss.item() if isinstance(total_loss, torch.Tensor) else total_loss
                    print(f" [Step {self.state.global_step:4d}] Gen: {gen_val:.4f} | CL: {cl_val:.4f} | Total: {total_val:.4f}")
            else:
                total_loss = gen_loss
        else:
            total_loss = gen_loss
        
        return (total_loss, outputs) if return_outputs else total_loss

# =====================================================================
# Training Arguments
# =====================================================================

training_args = TrainingArguments(
    output_dir=OUTPUT_MODEL_DIR,
    
    # ✅ FIXED: Lower learning rate
    learning_rate=2e-5,
    
    # ✅ FIXED: Better LR schedule
    lr_scheduler_type="cosine",
    warmup_steps=100,
    
    # Training params
    num_train_epochs=10,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4,
    
    # ✅ Validation & early stopping
    eval_strategy="steps",
    eval_steps=100,
    save_strategy="steps",
    save_steps=100,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    
    # Logging
    logging_dir=f"{OUTPUT_MODEL_DIR}/logs",
    logging_steps=25,
    logging_first_step=True,
    report_to="none",
    
    # Memory optimization
    fp16=True,
    dataloader_pin_memory=True,
    dataloader_num_workers=2,
    
    # Other
    seed=42,
)

# =====================================================================
# Early Stopping
# =====================================================================

early_stopping = EarlyStoppingCallback(
    early_stopping_patience=3,
    early_stopping_threshold=0.01
)

# =====================================================================
# Initialize Trainer
# =====================================================================

trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset_tokenized,
    eval_dataset=val_dataset_tokenized,
    data_collator=data_collator,
    callbacks=[early_stopping],
)

# =====================================================================
# Train
# =====================================================================

print(f"\n🚀 Starting training with:")
print(f"  CL Weight: {CL_WEIGHT}")
print(f"  Learning Rate: {training_args.learning_rate}")
print(f"  Batch Size: {training_args.per_device_train_batch_size}")
print(f"  Gradient Accumulation: {training_args.gradient_accumulation_steps}")
print(f"  Effective Batch Size: {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")
print(f"  Epochs: {training_args.num_train_epochs}")
print(f"  Early Stopping: patience=3")
print(f"  Dataset: {len(train_dataset_tokenized)} train, {len(val_dataset_tokenized)} val")
print()

start_time = time.time()
train_result = trainer.train()
training_time = time.time() - start_time

print("\n" + "="*70)
print("TRAINING COMPLETE")
print("="*70)
print(f"Training time: {training_time/60:.1f} minutes ({training_time/3600:.2f} hours)")
print(f"Final train loss: {train_result.metrics.get('train_loss', 'N/A'):.4f}")
print("="*70)

# Save model
trainer.save_model(OUTPUT_MODEL_DIR)
tokenizer.save_pretrained(OUTPUT_MODEL_DIR)
print(f"\n✓ Model and tokenizer saved to {OUTPUT_MODEL_DIR}")

# Print training metrics
print("\n" + "="*70)
print("***** train metrics *****")
for key, value in train_result.metrics.items():
    print(f"  {key:30s} = {value}")
print("="*70)

print("\n✅ CELL 8 COMPLETE - Proceed to Cell 9 (Generate Predictions)")


CELL 8: MODEL TRAINING (FIXED)

🚀 Starting training with:
  CL Weight: 0.05
  Learning Rate: 2e-05
  Batch Size: 4
  Gradient Accumulation: 4
  Effective Batch Size: 16
  Epochs: 10
  Early Stopping: patience=3
  Dataset: 2114 train, 452 val

 [Step    0] Gen: 13.7908 | CL: 2.0572 | Total: 13.8937
 [Step    0] Gen: 11.5351 | CL: 2.0569 | Total: 11.6380
 [Step    0] Gen: 13.2224 | CL: 2.0643 | Total: 13.3257
 [Step    0] Gen: 12.3273 | CL: 2.0594 | Total: 12.4303


Step,Training Loss,Validation Loss
100,2.642600,2.184846
200,2.134300,1.854277
300,2.038500,1.783376
400,1.968900,1.746238
500,1.924500,1.729986
600,1.895400,1.724814


 [Step   25] Gen: 5.4569 | CL: 2.0619 | Total: 5.5600
 [Step   25] Gen: 5.1518 | CL: 2.0591 | Total: 5.2548
 [Step   25] Gen: 5.2486 | CL: 2.0600 | Total: 5.3516
 [Step   25] Gen: 5.3645 | CL: 2.0629 | Total: 5.4677
 [Step   50] Gen: 3.6234 | CL: 2.0541 | Total: 3.7261
 [Step   50] Gen: 3.7596 | CL: 2.0560 | Total: 3.8624
 [Step   50] Gen: 3.5198 | CL: 2.0577 | Total: 3.6226
 [Step   50] Gen: 3.6579 | CL: 2.0527 | Total: 3.7605
 [Step   75] Gen: 2.7112 | CL: 2.0524 | Total: 2.8138
 [Step   75] Gen: 3.1364 | CL: 2.0583 | Total: 3.2393
 [Step   75] Gen: 2.6447 | CL: 2.0557 | Total: 2.7475
 [Step   75] Gen: 2.8371 | CL: 2.0531 | Total: 2.9397
 [Step  100] Gen: 2.1519 | CL: 2.0794 | Total: 2.2559
 [Step  100] Gen: 1.9215 | CL: 2.0794 | Total: 2.0254
 [Step  100] Gen: 2.1955 | CL: 2.0794 | Total: 2.2995
 [Step  100] Gen: 1.9878 | CL: 2.0794 | Total: 2.0918
 [Step  100] Gen: 1.9653 | CL: 2.0794 | Total: 2.0693
 [Step  100] Gen: 2.0106 | CL: 2.0794 | Total: 2.1145
 [Step  100] Gen: 1.8777 | C

There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight'].



TRAINING COMPLETE
Training time: 48.4 minutes (0.81 hours)
Final train loss: 2.4391

✓ Model and tokenizer saved to /kaggle/working/soft-cl-fewshot-absa

***** train metrics *****
  train_runtime                  = 2904.6404
  train_samples_per_second       = 7.278
  train_steps_per_second         = 0.231
  total_flos                     = 1.447577043075072e+16
  train_loss                     = 2.4390745589982217
  epoch                          = 10.0

✅ CELL 8 COMPLETE - Proceed to Cell 9 (Generate Predictions)


In [12]:
# =======================================================================
# CELL 9-12 COMBINED: COMPLETE EVALUATION PIPELINE
# =======================================================================

print("="*80)
print("COMPLETE EVALUATION PIPELINE")
print("="*80)

import json
import numpy as np
import pandas as pd
from tqdm import tqdm
from difflib import SequenceMatcher
from collections import Counter
import random

# =====================================================================
# STEP 1: GENERATE PREDICTIONS
# =====================================================================

print("\n" + "="*80)
print("STEP 1: GENERATING PREDICTIONS")
print("="*80)

model.eval()

test_predictions = []
print(f"\nGenerating predictions for {len(test_ds)} test samples...")

for idx, item in enumerate(tqdm(test_ds, desc="Generating")):
    input_text = item['input']
    gold_target = item['target']
    
    # Tokenize input
    inputs = tokenizer(
        input_text,
        max_length=512,
        truncation=True,
        padding=True,
        return_tensors="pt"
    ).to(model.device)
    
    # Generate
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=256,
            num_beams=4,
            early_stopping=True,
            no_repeat_ngram_size=2,
            temperature=0.9,
        )
    
    # Decode
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    test_predictions.append({
        'index': idx,
        'input': input_text,
        'gold_target': gold_target,
        'gen_output': generated_text
    })

# Save predictions
PRED_FILE = OUTPUT_MODEL_DIR / "test_predictions.jsonl"
with open(PRED_FILE, 'w') as f:
    for pred in test_predictions:
        f.write(json.dumps(pred) + '\n')

print(f"\n✓ Predictions saved to {PRED_FILE}")

# Show samples
print(f"\n" + "="*80)
print("SAMPLE PREDICTIONS (First 3)")
print("="*80)
for i in range(min(3, len(test_predictions))):
    item = test_predictions[i]
    print(f"\n--- Sample {i+1} ---")
    print(f"INPUT:\n {item['input'][:150]}...")
    print(f"\nGOLD:\n {item['gold_target']}")
    print(f"\nGENERATED:\n {item['gen_output']}")

# =====================================================================
# HELPER FUNCTIONS
# =====================================================================

def normalize_aspect(s):
    """Normalize aspect term."""
    if not s:
        return ''
    return re.sub(r'\s+', ' ', s.strip().lower())

def normalize_sentiment(s):
    """Normalize sentiment to positive/negative/neutral."""
    if not s:
        return ''
    s = s.strip().lower()
    if s.startswith('pos'):
        return 'positive'
    if s.startswith('neg'):
        return 'negative'
    if s.startswith('neu'):
        return 'neutral'
    return s

def parse_7field_output(text):
    """Parse 7-field format: aspect | sentiment | span | rationale | is_implicit | has_slang | has_emoji"""
    pairs = set()
    if not text or not text.strip():
        return pairs
    
    entries = re.split(r'\s*;\s*', text.strip())
    
    for entry in entries:
        fields = [f.strip() for f in entry.split('|')]
        
        if len(fields) >= 2:
            aspect = normalize_aspect(fields[0])
            sentiment = normalize_sentiment(fields[1])
            
            if aspect and sentiment in ['positive', 'negative', 'neutral']:
                pairs.add((aspect, sentiment))
    
    return pairs

def parse_7field_robust(text):
    """Parse 7-field format and return list of dicts."""
    result = []
    if not text or not text.strip():
        return result
    
    entries = re.split(r'\s*;\s*', text.strip())
    
    for entry in entries:
        fields = [f.strip() for f in entry.split('|')]
        
        if len(fields) >= 2:
            aspect = fields[0].strip().lower()
            sentiment = fields[1].strip().lower()
            
            if sentiment.startswith('pos'):
                sentiment = 'positive'
            elif sentiment.startswith('neg'):
                sentiment = 'negative'
            elif sentiment.startswith('neu'):
                sentiment = 'neutral'
            
            if aspect and sentiment in ['positive', 'negative', 'neutral']:
                result.append({
                    'aspect': aspect,
                    'sentiment': sentiment,
                    'span': fields[2] if len(fields) > 2 else '',
                    'rationale': fields[3] if len(fields) > 3 else '',
                    'is_implicit': fields[4] if len(fields) > 4 else '',
                    'has_slang': fields[5] if len(fields) > 5 else '',
                    'has_emoji': fields[6] if len(fields) > 6 else ''
                })
    
    return result

def compute_prf(tp, fp, fn):
    """Compute Precision, Recall, F1."""
    p = tp / (tp + fp) if (tp + fp) > 0 else 0.0
    r = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    f = 2 * p * r / (p + r) if (p + r) > 0 else 0.0
    return p, r, f

# =====================================================================
# STEP 2: PRIMARY ABSA METRICS
# =====================================================================

print("\n" + "="*80)
print("STEP 2: PRIMARY ABSA METRICS")
print("="*80)

# Format adherence
valid_format = 0
malformed = 0
empty_outputs = 0
field_counts = []

for item in test_predictions:
    gen_output = item['gen_output'].strip()
    
    if not gen_output:
        empty_outputs += 1
        continue
    
    entries = re.split(r'\s*;\s*', gen_output)
    all_valid = True
    
    for entry in entries:
        fields = [f.strip() for f in entry.split('|')]
        field_counts.append(len(fields))
        
        if len(fields) != 7:
            all_valid = False
        elif len(fields) >= 2:
            sent = fields[1].strip().lower()
            if sent not in ['positive', 'negative', 'neutral']:
                all_valid = False
    
    if all_valid:
        valid_format += 1
    else:
        malformed += 1

format_adherence = valid_format / len(test_predictions) if test_predictions else 0

print(f"\n✅ Format Adherence:")
print(f" Valid 7-field format: {valid_format}/{len(test_predictions)} ({format_adherence*100:.1f}%)")
print(f" Malformed outputs: {malformed} ({malformed/len(test_predictions)*100:.1f}%)")
print(f" Empty outputs: {empty_outputs} ({empty_outputs/len(test_predictions)*100:.1f}%)")
if field_counts:
    print(f" Avg fields per entry: {np.mean(field_counts):.2f}")

# Primary metrics
TP = FP = FN = 0  # Aspect + Sentiment
TP_aspect = FP_aspect = FN_aspect = 0  # Aspect-only
exact_match = 0
sentiment_correct = 0
detected_aspects = 0

for item in test_predictions:
    gold_pairs = parse_7field_output(item['gold_target'])
    pred_pairs = parse_7field_output(item['gen_output'])
    
    # Tuple-level
    TP += len(gold_pairs & pred_pairs)
    FP += len(pred_pairs - gold_pairs)
    FN += len(gold_pairs - pred_pairs)
    
    # Exact match
    if gold_pairs == pred_pairs:
        exact_match += 1
    
    # Aspect-only
    gold_aspects = set(a for a, _ in gold_pairs)
    pred_aspects = set(a for a, _ in pred_pairs)
    
    TP_aspect += len(gold_aspects & pred_aspects)
    FP_aspect += len(pred_aspects - gold_aspects)
    FN_aspect += len(gold_aspects - pred_aspects)
    
    # Sentiment accuracy
    for asp in (gold_aspects & pred_aspects):
        detected_aspects += 1
        gold_sents = set(s for a, s in gold_pairs if a == asp)
        pred_sents = set(s for a, s in pred_pairs if a == asp)
        if gold_sents & pred_sents:
            sentiment_correct += 1

tuple_p, tuple_r, tuple_f1 = compute_prf(TP, FP, FN)
aspect_p, aspect_r, aspect_f1 = compute_prf(TP_aspect, FP_aspect, FN_aspect)
exact_acc = exact_match / len(test_predictions) if test_predictions else 0.0
sent_acc = sentiment_correct / detected_aspects if detected_aspects > 0 else 0.0

print(f"\n✅ Primary Metrics:")
print(f" Aspect + Sentiment F1: {tuple_f1:.4f} (P: {tuple_p:.4f}, R: {tuple_r:.4f})")
print(f" Aspect-Only F1: {aspect_f1:.4f} (P: {aspect_p:.4f}, R: {aspect_r:.4f})")
print(f" Exact Match Accuracy: {exact_acc:.4f} ({exact_acc*100:.2f}%)")
print(f" Sentiment Accuracy: {sent_acc:.4f} ({sent_acc*100:.2f}%)")

# =====================================================================
# STEP 3: TEXT GENERATION QUALITY METRICS
# =====================================================================

print("\n" + "="*80)
print("STEP 3: TEXT GENERATION QUALITY METRICS")
print("="*80)

from bert_score import score as bert_score
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from sentence_transformers import SentenceTransformer
import nltk
nltk.download('punkt', quiet=True)

# BLEU
print("\n[1/4] Computing BLEU scores...")

def compute_bleu(reference, hypothesis):
    ref_tokens = reference.lower().split()
    hyp_tokens = hypothesis.lower().split()
    if not hyp_tokens:
        return 0.0
    smooth = SmoothingFunction().method1
    return sentence_bleu([ref_tokens], hyp_tokens, weights=(0.25, 0.25, 0.25, 0.25), smoothing_function=smooth)

bleu_scores = []
for item in test_predictions:
    gold = item['gold_target'].strip()
    pred = item['gen_output'].strip()
    if gold and pred:
        bleu_scores.append(compute_bleu(gold, pred))

avg_bleu = np.mean(bleu_scores) if bleu_scores else 0.0
print(f" BLEU-4: {avg_bleu:.4f}")

# ROUGE
print("\n[2/4] Computing ROUGE scores...")

rouge_scorer_obj = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
rouge1_f1 = []
rouge2_f1 = []
rougeL_f1 = []

for item in test_predictions:
    gold = item['gold_target'].strip()
    pred = item['gen_output'].strip()
    if gold and pred:
        scores = rouge_scorer_obj.score(gold, pred)
        rouge1_f1.append(scores['rouge1'].fmeasure)
        rouge2_f1.append(scores['rouge2'].fmeasure)
        rougeL_f1.append(scores['rougeL'].fmeasure)

avg_r1 = np.mean(rouge1_f1) if rouge1_f1 else 0.0
avg_r2 = np.mean(rouge2_f1) if rouge2_f1 else 0.0
avg_rl = np.mean(rougeL_f1) if rougeL_f1 else 0.0
print(f" ROUGE-1: {avg_r1:.4f}, ROUGE-2: {avg_r2:.4f}, ROUGE-L: {avg_rl:.4f}")

# BERTScore
print("\n[3/4] Computing BERTScore...")

gold_texts = [item['gold_target'].strip() for item in test_predictions if item['gold_target'].strip()]
pred_texts = [item['gen_output'].strip() for item in test_predictions if item['gold_target'].strip()]

if gold_texts and pred_texts:
    P, R, F1 = bert_score(pred_texts, gold_texts, lang='en', verbose=False)
    avg_bert_f1 = F1.mean().item()
    print(f" BERTScore F1: {avg_bert_f1:.4f}")
else:
    avg_bert_f1 = 0.0

# Semantic Similarity
print("\n[4/4] Computing semantic similarity...")

semantic_model = SentenceTransformer('all-MiniLM-L6-v2')
cosine_sims = []

for item in test_predictions:
    gold = item['gold_target'].strip()
    pred = item['gen_output'].strip()
    if gold and pred:
        gold_emb = semantic_model.encode(gold, convert_to_tensor=True)
        pred_emb = semantic_model.encode(pred, convert_to_tensor=True)
        sim = torch.nn.functional.cosine_similarity(gold_emb.unsqueeze(0), pred_emb.unsqueeze(0)).item()
        cosine_sims.append(sim)

avg_cosine = np.mean(cosine_sims) if cosine_sims else 0.0
print(f" Cosine Similarity: {avg_cosine:.4f}")

# =====================================================================
# STEP 4: ERROR ANALYSIS
# =====================================================================

print("\n" + "="*80)
print("STEP 4: ERROR ANALYSIS")
print("="*80)

# Span matching
def span_match_ratio(gold, pred):
    return SequenceMatcher(None, gold.lower(), pred.lower()).ratio()

span_matches = []
for item in test_predictions:
    gold = item['gold_target'].strip()
    pred = item['gen_output'].strip()
    if gold and pred:
        span_matches.append(span_match_ratio(gold, pred))

avg_span_match = np.mean(span_matches) if span_matches else 0.0

# Error categories
error_stats = {
    'perfect_match': 0,
    'minor_format_error': 0,
    'missing_fields': 0,
    'extra_aspects': 0,
    'wrong_sentiment': 0,
    'empty_output': 0,
    'parsing_error': 0
}

for item in test_predictions:
    gold_data = parse_7field_robust(item['gold_target'])
    pred_data = parse_7field_robust(item['gen_output'])
    
    if not item['gen_output'].strip():
        error_stats['empty_output'] += 1
    elif len(pred_data) == 0:
        error_stats['parsing_error'] += 1
    elif len(pred_data) == len(gold_data):
        gold_set = {(p['aspect'], p['sentiment']) for p in gold_data}
        pred_set = {(p['aspect'], p['sentiment']) for p in pred_data}
        
        if gold_set == pred_set:
            error_stats['perfect_match'] += 1
        else:
            gold_aspects = {p['aspect'] for p in gold_data}
            pred_aspects = {p['aspect'] for p in pred_data}
            if gold_aspects == pred_aspects:
                error_stats['wrong_sentiment'] += 1
            else:
                error_stats['minor_format_error'] += 1
    else:
        if len(pred_data) < len(gold_data):
            error_stats['missing_fields'] += 1
        else:
            error_stats['extra_aspects'] += 1

print(f"\n✅ Error Category Distribution:")
for category, count in error_stats.items():
    pct = count / len(test_predictions) * 100
    print(f" {category:25s}: {count:3d} ({pct:5.1f}%)")

# =====================================================================
# STEP 5: MODE COLLAPSE CHECK
# =====================================================================

print("\n" + "="*80)
print("STEP 5: MODE COLLAPSE CHECK")
print("="*80)

unique_predictions = {}
for item in test_predictions:
    pred = item['gen_output'].strip()
    unique_predictions[pred] = unique_predictions.get(pred, 0) + 1

print(f"\nTotal predictions: {len(test_predictions)}")
print(f"Unique predictions: {len(unique_predictions)}")
print(f"Repetition rate: {(1 - len(unique_predictions)/len(test_predictions))*100:.1f}%")

# Aspect diversity
predicted_aspects = []
for item in test_predictions:
    parsed = parse_7field_robust(item['gen_output'])
    for p in parsed:
        predicted_aspects.append(p['aspect'])

if predicted_aspects:
    aspect_counts = Counter(predicted_aspects)
    print(f"\nTotal predicted aspects: {len(predicted_aspects)}")
    print(f"Unique aspects: {len(aspect_counts)}")
    print(f"\nTop 10 most predicted aspects:")
    for aspect, count in aspect_counts.most_common(10):
        pct = count / len(predicted_aspects) * 100
        print(f"  {aspect:<30} : {count:3d} ({pct:5.1f}%)")

# =====================================================================
# STEP 6: COMPREHENSIVE RESULTS TABLE
# =====================================================================

print("\n" + "="*80)
print("COMPREHENSIVE EVALUATION RESULTS")
print("="*80)

results_df = pd.DataFrame({
    'Category': [
        'ABSA Metrics',
        '',
        '',
        'Text Quality',
        '',
        '',
        '',
        'Semantic',
        '',
        'Error Analysis',
        '',
        '',
        'Diversity'
    ],
    'Metric': [
        'Aspect+Sentiment F1',
        'Aspect-Only F1',
        'Exact Match %',
        'BLEU-4',
        'ROUGE-1 F1',
        'ROUGE-2 F1',
        'ROUGE-L F1',
        'BERTScore F1',
        'Cosine Similarity',
        'Perfect Matches',
        'Sentiment Errors',
        'Missing Fields',
        'Unique Predictions'
    ],
    'Value': [
        f"{tuple_f1:.4f}",
        f"{aspect_f1:.4f}",
        f"{exact_acc:.2%}",
        f"{avg_bleu:.4f}",
        f"{avg_r1:.4f}",
        f"{avg_r2:.4f}",
        f"{avg_rl:.4f}",
        f"{avg_bert_f1:.4f}",
        f"{avg_cosine:.4f}",
        f"{error_stats['perfect_match']} ({error_stats['perfect_match']/len(test_predictions)*100:.1f}%)",
        f"{error_stats['wrong_sentiment']}",
        f"{error_stats['missing_fields']}",
        f"{len(unique_predictions)} / {len(test_predictions)}"
    ]
})

print(results_df.to_string(index=False))

# =====================================================================
# STEP 7: SAVE FINAL REPORT
# =====================================================================

final_report = {
    "dataset": {
        "total_samples": len(test_predictions),
        "valid_predictions": len([x for x in test_predictions if x['gen_output'].strip()])
    },
    "absa_metrics": {
        "aspect_sentiment_f1": float(tuple_f1),
        "aspect_only_f1": float(aspect_f1),
        "exact_match": float(exact_acc),
        "format_adherence": float(format_adherence)
    },
    "text_quality": {
        "bleu4": float(avg_bleu),
        "rouge1_f1": float(avg_r1),
        "rouge2_f1": float(avg_r2),
        "rougeL_f1": float(avg_rl)
    },
    "semantic_metrics": {
        "bertscore_f1": float(avg_bert_f1),
        "cosine_similarity": float(avg_cosine),
        "span_match_ratio": float(avg_span_match)
    },
    "error_analysis": error_stats,
    "diversity": {
        "unique_predictions": len(unique_predictions),
        "total_predictions": len(test_predictions),
        "repetition_rate": float((1 - len(unique_predictions)/len(test_predictions)))
    }
}

REPORT_FILE = OUTPUT_MODEL_DIR / "final_evaluation_report.json"
with open(REPORT_FILE, 'w') as f:
    json.dump(final_report, f, indent=2)

print(f"\n✓ Final report saved to {REPORT_FILE}")

# =====================================================================
# STEP 8: RANDOM SAMPLE INSPECTION
# =====================================================================

print("\n" + "="*80)
print("RANDOM SAMPLE INSPECTION")
print("="*80)

num_samples = min(5, len(test_predictions))
sample_indices = random.sample(range(len(test_predictions)), num_samples)

for i, idx in enumerate(sample_indices, 1):
    item = test_predictions[idx]
    
    print(f"\n{'='*80}")
    print(f"SAMPLE {i} (Index: {idx})")
    print(f"{'='*80}")
    
    print(f"\n📝 INPUT: {item['input'][:150]}...")
    
    gold_parsed = parse_7field_robust(item['gold_target'])
    pred_parsed = parse_7field_robust(item['gen_output'])
    
    print(f"\n✅ GOLD ({len(gold_parsed)} aspects): {item['gold_target'][:100]}...")
    print(f"\n🤖 PRED ({len(pred_parsed)} aspects): {item['gen_output'][:100]}...")
    
    print(f"\n📊 COMPARISON:")
    max_len = max(len(gold_parsed), len(pred_parsed))
    for j in range(max_len):
        gold_asp = gold_parsed[j]['aspect'] if j < len(gold_parsed) else "—"
        gold_sent = gold_parsed[j]['sentiment'] if j < len(gold_parsed) else "—"
        pred_asp = pred_parsed[j]['aspect'] if j < len(pred_parsed) else "—"
        pred_sent = pred_parsed[j]['sentiment'] if j < len(pred_parsed) else "—"
        
        match = "✅" if (gold_asp == pred_asp and gold_sent == pred_sent) else "❌"
        print(f"{match} Aspect: {gold_asp:<20} → {pred_asp:<20}")
        print(f"   Sentiment: {gold_sent:<10} → {pred_sent:<10}")

print("\n" + "="*80)
print("✅ COMPLETE EVALUATION FINISHED")
print("="*80)


COMPLETE EVALUATION PIPELINE

STEP 1: GENERATING PREDICTIONS

Generating predictions for 453 test samples...


Generating: 100%|██████████| 453/453 [06:02<00:00,  1.25it/s]



✓ Predictions saved to /kaggle/working/soft-cl-fewshot-absa/test_predictions.jsonl

SAMPLE PREDICTIONS (First 3)

--- Sample 1 ---
INPUT:
 Extract aspects, sentiments, and rationales. Output format: aspect | sentiment | span | rationale | is_implicit | has_slang | has_emoji
Separate multi...

GOLD:
 election_fraud | negative | it's rigged, unless I win | explicitly uses 'it's rigged, unless I win' to express negative sentiment about election_fraud claims by mocking Republican hypocrisy. | FALSE | FALSE | FALSE

GENERATED:
 service_speed | negative | service | Explicit negative: 'service speed sucks' slow. | FALSE | TRUE | TRULY

--- Sample 2 ---
INPUT:
 Extract aspects, sentiments, and rationales. Output format: aspect | sentiment | span | rationale | is_implicit | has_slang | has_emoji
Separate multi...

GOLD:
 interface | positive | interface | Explicit positive: 'interface gesture natural' intuitive. | FALSE | FALSE | FALSE

GENERATED:
 service_speed | negative | service | Explicit

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

 Cosine Similarity: 0.3294

STEP 4: ERROR ANALYSIS

✅ Error Category Distribution:
 perfect_match            :   4 (  0.9%)
 minor_format_error       : 331 ( 73.1%)
 missing_fields           : 117 ( 25.8%)
 extra_aspects            :   0 (  0.0%)
 wrong_sentiment          :   1 (  0.2%)
 empty_output             :   0 (  0.0%)
 parsing_error            :   0 (  0.0%)

STEP 5: MODE COLLAPSE CHECK

Total predictions: 453
Unique predictions: 1
Repetition rate: 99.8%

Total predicted aspects: 453
Unique aspects: 1

Top 10 most predicted aspects:
  service_speed                  : 453 (100.0%)

COMPREHENSIVE EVALUATION RESULTS
      Category              Metric    Value
  ABSA Metrics Aspect+Sentiment F1   0.0326
                    Aspect-Only F1   0.0364
                     Exact Match %    0.88%
  Text Quality              BLEU-4   0.0455
                        ROUGE-1 F1   0.1442
                        ROUGE-2 F1   0.0165
                        ROUGE-L F1   0.1283
      Semantic    

In [13]:
# # =======================================================================
# # CELL 9: GENERATE PREDICTIONS ON TEST SET
# # =======================================================================

# print("="*70)
# print("CELL 9: GENERATING PREDICTIONS")
# print("="*70)

# # Use trained model
# print(f"Using trained model from {OUTPUT_MODEL_DIR}...")
# model.eval()

# # Ensure decoder_start_token_id is set
# if model.config.decoder_start_token_id is None:
#     model.config.decoder_start_token_id = tokenizer.pad_token_id
#     print(f"✓ Set decoder_start_token_id to {tokenizer.pad_token_id}")

# def generate_predictions(dataset, tokenizer, model, device, batch_size=PER_DEVICE_EVAL_BATCH_SIZE):
#     """Generate predictions with progress bar."""
#     all_preds = []
    
#     for i in tqdm(range(0, len(dataset), batch_size), desc="Generating"):
#         batch = dataset[i:i + batch_size]
#         input_texts = batch['input']
#         gold_targets = batch['target']
        
#         # Tokenize
#         inputs = tokenizer(
#             input_texts,
#             return_tensors="pt",
#             padding=True,
#             truncation=True,
#             max_length=MAX_INPUT_LENGTH
#         ).to(device)
        
#         # Generate
#         with torch.no_grad():
#             generated_ids = model.generate(
#                 **inputs,
#                 num_beams=GEN_NUM_BEAMS,
#                 max_length=MAX_TARGET_LENGTH,
#                 early_stopping=True,
#                 decoder_start_token_id=model.config.decoder_start_token_id
#             )
        
#         # Decode
#         generated_texts = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
        
#         # Store results
#         for j in range(len(input_texts)):
#             all_preds.append({
#                 'input': input_texts[j],
#                 'gold_target': gold_targets[j],
#                 'gen_output': generated_texts[j]
#             })
        
#         return all_preds

# # Generate predictions
# print(f"\nGenerating predictions on {len(test_ds)} test samples...\n")
# test_predictions = generate_predictions(test_ds, tokenizer, model, DEVICE)

# # Save predictions
# PRED_FILE = OUTPUT_MODEL_DIR / "test_predictions.jsonl"
# with open(PRED_FILE, 'w', encoding='utf-8') as f:
#     for pred in test_predictions:
#         f.write(json.dumps(pred, ensure_ascii=False) + '\n')

# print(f"\n✓ Predictions saved to {PRED_FILE}")

# # Show sample predictions
# print("\n" + "="*70)
# print("SAMPLE PREDICTIONS (First 3)")
# print("="*70)

# for i in range(min(3, len(test_predictions))):
#     pred = test_predictions[i]
#     print(f"\n--- Sample {i+1} ---")
#     print(f"GOLD:")
#     print(f" {pred['gold_target']}")
#     print(f"GENERATED:")
#     print(f" {pred['gen_output']}")

# print("\n" + "="*70)
# print("✅ CELL 9 COMPLETE - Proceed to Cell 10 (Evaluation)")
# print("="*70)


In [14]:
# # =======================================================================
# # CELL 10: COMPREHENSIVE EVALUATION METRICS
# # =======================================================================

# print("="*70)
# print("CELL 10: COMPREHENSIVE EVALUATION")
# print("="*70)

# # =======================================================================
# # PARSING FUNCTIONS
# # =======================================================================

# def normalize_aspect(s):
#     """Normalize aspect term."""
#     if not s:
#         return ''
#     return re.sub(r'\s+', ' ', s.strip().lower())

# def normalize_sentiment(s):
#     """Normalize sentiment to positive/negative/neutral."""
#     if not s:
#         return ''
#     s = s.strip().lower()
#     if s.startswith('pos'):
#         return 'positive'
#     if s.startswith('neg'):
#         return 'negative'
#     if s.startswith('neu'):
#         return 'neutral'
#     return s

# def parse_7field_output(text):
#     """
#     Parse 7-field format: aspect | sentiment | span | rationale | is_implicit | has_slang | has_emoji
#     Returns: set of (aspect, sentiment) tuples for matching
#     """
#     pairs = set()
#     if not text or not text.strip():
#         return pairs
    
#     # Split by ' ; ' separator
#     entries = re.split(r'\s*;\s*', text.strip())
    
#     for entry in entries:
#         # Split by ' | ' to get fields
#         fields = [f.strip() for f in entry.split('|')]
        
#         if len(fields) >= 2:
#             aspect = normalize_aspect(fields[0])      # ✅ FIX 1: fields[0]
#             sentiment = normalize_sentiment(fields[1])  # ✅ FIX 2: fields[1]
            
#             if aspect and sentiment in ['positive', 'negative', 'neutral']:
#                 pairs.add((aspect, sentiment))
    
#     return pairs

# def compute_prf(tp, fp, fn):
#     """Compute Precision, Recall, F1."""
#     p = tp / (tp + fp) if (tp + fp) > 0 else 0.0
#     r = tp / (tp + fn) if (tp + fn) > 0 else 0.0
#     f = 2 * p * r / (p + r) if (p + r) > 0 else 0.0
#     return p, r, f

# # =======================================================================
# # FORMAT ADHERENCE CHECK
# # =======================================================================

# print("\n[1/4] Checking format adherence...")

# valid_format = 0
# malformed = 0
# empty_outputs = 0
# field_counts = []

# for item in test_predictions:
#     gen_output = item['gen_output'].strip()
    
#     if not gen_output:
#         empty_outputs += 1
#         continue
    
#     entries = re.split(r'\s*;\s*', gen_output)
#     all_valid = True
    
#     for entry in entries:
#         fields = [f.strip() for f in entry.split('|')]
#         field_counts.append(len(fields))
        
#         if len(fields) != 7:
#             all_valid = False
#         elif len(fields) >= 2:
#             sent = fields[1].strip().lower()  # ✅ FIX 3: fields[1]
#             if sent not in ['positive', 'negative', 'neutral']:
#                 all_valid = False
    
#     if all_valid:
#         valid_format += 1
#     else:
#         malformed += 1

# format_adherence = valid_format / len(test_predictions) if test_predictions else 0

# print(f"\n✅ Format Adherence:")
# print(f" Valid 7-field format: {valid_format}/{len(test_predictions)} ({format_adherence*100:.1f}%)")
# print(f" Malformed outputs: {malformed} ({malformed/len(test_predictions)*100:.1f}%)")
# print(f" Empty outputs: {empty_outputs} ({empty_outputs/len(test_predictions)*100:.1f}%)")
# if field_counts:
#     print(f" Avg fields per entry: {np.mean(field_counts):.2f}")

# # =======================================================================
# # PRIMARY METRICS
# # =======================================================================

# print("\n[2/4] Computing primary metrics...")

# TP = FP = FN = 0  # Aspect + Sentiment
# TP_aspect = FP_aspect = FN_aspect = 0  # Aspect-only
# exact_match = 0
# sentiment_correct = 0
# detected_aspects = 0

# for item in test_predictions:
#     gold_pairs = parse_7field_output(item['gold_target'])
#     pred_pairs = parse_7field_output(item['gen_output'])
    
#     # Tuple-level (aspect + sentiment)
#     TP += len(gold_pairs & pred_pairs)
#     FP += len(pred_pairs - gold_pairs)
#     FN += len(gold_pairs - pred_pairs)
    
#     # Exact match
#     if gold_pairs == pred_pairs:
#         exact_match += 1
    
#     # Aspect-only
#     gold_aspects = set(a for a, _ in gold_pairs)
#     pred_aspects = set(a for a, _ in pred_pairs)
    
#     TP_aspect += len(gold_aspects & pred_aspects)
#     FP_aspect += len(pred_aspects - gold_aspects)
#     FN_aspect += len(gold_aspects - pred_aspects)
    
#     # Sentiment accuracy on detected aspects
#     for asp in (gold_aspects & pred_aspects):
#         detected_aspects += 1
#         gold_sents = set(s for a, s in gold_pairs if a == asp)
#         pred_sents = set(s for a, s in pred_pairs if a == asp)
#         if gold_sents & pred_sents:
#             sentiment_correct += 1

# tuple_p, tuple_r, tuple_f1 = compute_prf(TP, FP, FN)
# aspect_p, aspect_r, aspect_f1 = compute_prf(TP_aspect, FP_aspect, FN_aspect)
# exact_acc = exact_match / len(test_predictions) if test_predictions else 0.0
# sent_acc = sentiment_correct / detected_aspects if detected_aspects > 0 else 0.0

# print(f"\n✅ Primary Metrics:")
# print(f" Aspect + Sentiment F1: {tuple_f1:.4f} (P: {tuple_p:.4f}, R: {tuple_r:.4f})")
# print(f" Aspect-Only F1: {aspect_f1:.4f} (P: {aspect_p:.4f}, R: {aspect_r:.4f})")
# print(f" Exact Match Accuracy: {exact_acc:.4f} ({exact_acc*100:.2f}%)")
# print(f" Sentiment Accuracy: {sent_acc:.4f} ({sent_acc*100:.2f}%)")

# # =======================================================================
# # SUMMARY & FINAL REPORT
# # =======================================================================

# print("\n" + "="*70)
# print("FINAL EVALUATION SUMMARY")
# print("="*70)

# print(f"\n📊 Dataset Performance:")
# print(f" Test samples: {len(test_predictions)}")
# print(f" Valid predictions: {valid_format}/{len(test_predictions)} ({format_adherence*100:.1f}%)")
# print(f" Exact matches: {exact_match}/{len(test_predictions)} ({exact_acc*100:.2f}%)")

# print(f"\n📈 Aspect-Based Sentiment Analysis Metrics:")
# print(f" Aspect + Sentiment F1:  {tuple_f1:.4f}")
# print(f"  ├─ Precision: {tuple_p:.4f}")
# print(f"  ├─ Recall:    {tuple_r:.4f}")
# print(f"  └─ TP: {TP}, FP: {FP}, FN: {FN}")
# print(f"\n Aspect-Only F1:         {aspect_f1:.4f}")
# print(f"  ├─ Precision: {aspect_p:.4f}")
# print(f"  ├─ Recall:    {aspect_r:.4f}")
# print(f"  └─ TP: {TP_aspect}, FP: {FP_aspect}, FN: {FN_aspect}")

# print(f"\n💡 Sentiment Understanding:")
# print(f" Sentiment Accuracy:     {sent_acc:.4f} ({sent_acc*100:.2f}%)")
# print(f"  ├─ Detected aspects: {detected_aspects}")
# print(f"  ├─ Correct sentiments: {sentiment_correct}")
# print(f"  └─ Accuracy on correct aspects: {sent_acc:.1%}")

# print(f"\n🎯 Format Quality:")
# print(f" Valid format: {format_adherence:.1%} ({valid_format}/{len(test_predictions)})")
# print(f" Malformed: {malformed} ({malformed/len(test_predictions)*100:.1f}%)")
# print(f" Empty: {empty_outputs} ({empty_outputs/len(test_predictions)*100:.1f}%)")

# # Save results
# RESULTS_FILE = OUTPUT_MODEL_DIR / "evaluation_results.json"
# results = {
#     "total_samples": len(test_predictions),
#     "valid_format": valid_format,
#     "format_adherence_pct": format_adherence * 100,
#     "exact_match_count": exact_match,
#     "exact_match_pct": exact_acc * 100,
#     "aspect_sentiment_f1": tuple_f1,
#     "aspect_sentiment_precision": tuple_p,
#     "aspect_sentiment_recall": tuple_r,
#     "aspect_only_f1": aspect_f1,
#     "aspect_only_precision": aspect_p,
#     "aspect_only_recall": aspect_r,
#     "sentiment_accuracy": sent_acc,
#     "sentiment_accuracy_pct": sent_acc * 100
# }

# with open(RESULTS_FILE, 'w') as f:
#     json.dump(results, f, indent=2)

# print(f"\n✓ Results saved to {RESULTS_FILE}")

# print("\n" + "="*70)
# print("✅ CELL 10 COMPLETE - Proceed to Cell 11 (Text Quality Metrics)")
# print("="*70)


In [15]:
# # =======================================================================
# # CELL 11: ADVANCED TEXT GENERATION QUALITY METRICS
# # =======================================================================

# print("="*70)
# print("CELL 11: TEXT GENERATION QUALITY METRICS")
# print("="*70)

# from bert_score import score as bert_score
# from rouge_score import rouge_scorer
# from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
# import nltk
# nltk.download('punkt', quiet=True)

# # =====================================================================
# # 1. BLEU SCORE (Precision-based n-gram overlap)
# # =====================================================================

# print("\n[1/4] Computing BLEU scores...")

# def compute_bleu(reference, hypothesis):
#     """Compute BLEU-4 score."""
#     ref_tokens = reference.lower().split()
#     hyp_tokens = hypothesis.lower().split()
    
#     if not hyp_tokens:
#         return 0.0
    
#     smooth = SmoothingFunction().method1
#     return sentence_bleu(
#         [ref_tokens],
#         hyp_tokens,
#         weights=(0.25, 0.25, 0.25, 0.25),
#         smoothing_function=smooth
#     )

# bleu_scores = []
# for item in test_predictions:
#     gold = item['gold_target'].strip()
#     pred = item['gen_output'].strip()
    
#     if gold and pred:
#         bleu = compute_bleu(gold, pred)
#         bleu_scores.append(bleu)

# avg_bleu = np.mean(bleu_scores) if bleu_scores else 0.0
# median_bleu = np.median(bleu_scores) if bleu_scores else 0.0

# print(f"\n✅ BLEU Scores:")
# print(f" Mean BLEU-4: {avg_bleu:.4f}")
# print(f" Median BLEU-4: {median_bleu:.4f}")
# print(f" Std Dev: {np.std(bleu_scores):.4f}")
# print(f" Range: [{np.min(bleu_scores):.4f}, {np.max(bleu_scores):.4f}]")

# # =====================================================================
# # 2. ROUGE SCORES (Recall-based n-gram + LCS overlap)
# # =====================================================================

# print("\n[2/4] Computing ROUGE scores...")

# rouge_scorer_obj = rouge_scorer.RougeScorer(
#     ['rouge1', 'rouge2', 'rougeL'], 
#     use_stemmer=True
# )

# rouge1_f1 = []
# rouge2_f1 = []
# rougeL_f1 = []

# for item in test_predictions:
#     gold = item['gold_target'].strip()
#     pred = item['gen_output'].strip()
    
#     if gold and pred:
#         scores = rouge_scorer_obj.score(gold, pred)
        
#         rouge1_f1.append(scores['rouge1'].fmeasure)
#         rouge2_f1.append(scores['rouge2'].fmeasure)
#         rougeL_f1.append(scores['rougeL'].fmeasure)

# avg_r1 = np.mean(rouge1_f1) if rouge1_f1 else 0.0
# avg_r2 = np.mean(rouge2_f1) if rouge2_f1 else 0.0
# avg_rl = np.mean(rougeL_f1) if rougeL_f1 else 0.0

# print(f"\n✅ ROUGE Scores (F1 Measure):")
# print(f" ROUGE-1 (Unigram): {avg_r1:.4f}")
# print(f" ROUGE-2 (Bigram):  {avg_r2:.4f}")
# print(f" ROUGE-L (LCS):     {avg_rl:.4f}")

# # =====================================================================
# # 3. BERTScore (Contextual embedding similarity)
# # =====================================================================

# print("\n[3/4] Computing BERTScore...")

# gold_texts = [item['gold_target'].strip() for item in test_predictions if item['gold_target'].strip()]
# pred_texts = [item['gen_output'].strip() for item in test_predictions if item['gold_target'].strip()]

# if gold_texts and pred_texts:
#     P, R, F1 = bert_score(pred_texts, gold_texts, lang='en', verbose=False)
    
#     avg_bert_p = P.mean().item()
#     avg_bert_r = R.mean().item()
#     avg_bert_f1 = F1.mean().item()
    
#     print(f"\n✅ BERTScore (contextual similarity):")
#     print(f" Precision: {avg_bert_p:.4f}")
#     print(f" Recall:    {avg_bert_r:.4f}")
#     print(f" F1:        {avg_bert_f1:.4f}")
# else:
#     avg_bert_p = avg_bert_r = avg_bert_f1 = 0.0
#     print(f"\n⚠️  Skipping BERTScore (no valid predictions)")

# # =====================================================================
# # 4. SEMANTIC SIMILARITY (using SentenceTransformer)
# # =====================================================================

# print("\n[4/4] Computing semantic similarity...")

# semantic_sim = SentenceTransformer('all-MiniLM-L6-v2')

# cosine_sims = []
# for item in tqdm(test_predictions, desc="Computing embeddings"):
#     gold = item['gold_target'].strip()
#     pred = item['gen_output'].strip()
    
#     if gold and pred:
#         gold_emb = semantic_sim.encode(gold, convert_to_tensor=True)
#         pred_emb = semantic_sim.encode(pred, convert_to_tensor=True)
        
#         sim = torch.nn.functional.cosine_similarity(
#             gold_emb.unsqueeze(0), 
#             pred_emb.unsqueeze(0)
#         ).item()
#         cosine_sims.append(sim)

# avg_cosine = np.mean(cosine_sims) if cosine_sims else 0.0

# print(f"\n✅ Semantic Similarity (Cosine):")
# print(f" Mean Similarity: {avg_cosine:.4f}")
# print(f" Range: [{np.min(cosine_sims):.4f}, {np.max(cosine_sims):.4f}]")

# # =====================================================================
# # SUMMARY TABLE
# # =====================================================================

# print("\n" + "="*70)
# print("TEXT GENERATION QUALITY SUMMARY")
# print("="*70)

# quality_metrics = pd.DataFrame({
#     'Metric': [
#         'BLEU-4 (Precision)',
#         'ROUGE-1 (Unigram F1)',
#         'ROUGE-2 (Bigram F1)',
#         'ROUGE-L (LCS F1)',
#         'BERTScore Precision',
#         'BERTScore Recall',
#         'BERTScore F1',
#         'Semantic Similarity (Cosine)'
#     ],
#     'Value': [
#         f"{avg_bleu:.4f}",
#         f"{avg_r1:.4f}",
#         f"{avg_r2:.4f}",
#         f"{avg_rl:.4f}",
#         f"{avg_bert_p:.4f}",
#         f"{avg_bert_r:.4f}",
#         f"{avg_bert_f1:.4f}",
#         f"{avg_cosine:.4f}"
#     ]
# })

# print(quality_metrics.to_string(index=False))
# print("="*70)

# print("\n✅ CELL 11 COMPLETE - Proceed to Cell 12 (Advanced Analysis)")


In [16]:
# # =======================================================================
# # CELL 12: ADVANCED ERROR ANALYSIS & BREAKDOWN
# # =======================================================================

# print("="*70)
# print("CELL 12: ADVANCED ERROR ANALYSIS")
# print("="*70)

# # =====================================================================
# # HELPER FUNCTION (if not already defined)
# # =====================================================================

# def parse_7field_robust(text):
#     """
#     Parse 7-field format and return list of dicts.
#     Format: aspect | sentiment | span | rationale | is_implicit | has_slang | has_emoji
#     """
#     result = []
#     if not text or not text.strip():
#         return result
    
#     entries = re.split(r'\s*;\s*', text.strip())
    
#     for entry in entries:
#         fields = [f.strip() for f in entry.split('|')]
        
#         if len(fields) >= 2:
#             aspect = fields[0].strip().lower()
#             sentiment = fields[1].strip().lower()
            
#             # Normalize sentiment
#             if sentiment.startswith('pos'):
#                 sentiment = 'positive'
#             elif sentiment.startswith('neg'):
#                 sentiment = 'negative'
#             elif sentiment.startswith('neu'):
#                 sentiment = 'neutral'
            
#             if aspect and sentiment in ['positive', 'negative', 'neutral']:
#                 result.append({
#                     'aspect': aspect,
#                     'sentiment': sentiment,
#                     'span': fields[2] if len(fields) > 2 else '',
#                     'rationale': fields[3] if len(fields) > 3 else '',
#                     'is_implicit': fields[4] if len(fields) > 4 else '',
#                     'has_slang': fields[5] if len(fields) > 5 else '',
#                     'has_emoji': fields[6] if len(fields) > 6 else ''
#                 })
    
#     return result

# # =====================================================================
# # 1. SPAN-LEVEL MATCHING & PARTIAL CREDIT
# # =====================================================================

# print("\n[1/4] Computing span-level matching...")

# from difflib import SequenceMatcher

# def span_match_ratio(gold, pred):
#     """Compute string matching ratio using SequenceMatcher."""
#     return SequenceMatcher(None, gold.lower(), pred.lower()).ratio()

# span_matches = []
# partial_credit = []

# for item in test_predictions:
#     gold = item['gold_target'].strip()
#     pred = item['gen_output'].strip()
    
#     if gold and pred:
#         ratio = span_match_ratio(gold, pred)
#         span_matches.append(ratio)
        
#         # Partial credit: if >50% similar
#         if ratio >= 0.5:
#             partial_credit.append(1)
#         else:
#             partial_credit.append(0)

# avg_span_match = np.mean(span_matches) if span_matches else 0.0
# partial_accuracy = np.mean(partial_credit) if partial_credit else 0.0

# print(f"\n✅ Span-Level Matching:")
# print(f" Avg String Match Ratio: {avg_span_match:.4f}")
# print(f" Partial Credit (≥50% match): {partial_accuracy:.2%}")

# # =====================================================================
# # 2. ERROR CATEGORY BREAKDOWN
# # =====================================================================

# print("\n[2/4] Categorizing errors...")

# error_stats = {
#     'perfect_match': 0,
#     'minor_format_error': 0,
#     'missing_fields': 0,
#     'extra_aspects': 0,
#     'wrong_sentiment': 0,
#     'empty_output': 0,
#     'parsing_error': 0
# }

# for item in test_predictions:
#     gold_data = parse_7field_robust(item['gold_target'])
#     pred_data = parse_7field_robust(item['gen_output'])
    
#     if not item['gen_output'].strip():
#         error_stats['empty_output'] += 1
#     elif len(pred_data) == 0:
#         error_stats['parsing_error'] += 1
#     elif len(pred_data) == len(gold_data):
#         gold_set = {(p['aspect'], p['sentiment']) for p in gold_data}
#         pred_set = {(p['aspect'], p['sentiment']) for p in pred_data}
        
#         if gold_set == pred_set:
#             error_stats['perfect_match'] += 1
#         else:
#             # Check what's different
#             gold_aspects = {p['aspect'] for p in gold_data}
#             pred_aspects = {p['aspect'] for p in pred_data}
            
#             if gold_aspects == pred_aspects:
#                 # Same aspects, different sentiments
#                 error_stats['wrong_sentiment'] += 1
#             else:
#                 error_stats['minor_format_error'] += 1
#     else:
#         if len(pred_data) < len(gold_data):
#             error_stats['missing_fields'] += 1
#         else:
#             error_stats['extra_aspects'] += 1

# total_samples = len(test_predictions)

# print(f"\n✅ Error Category Distribution:")
# for category, count in error_stats.items():
#     pct = count / total_samples * 100
#     print(f" {category:25s}: {count:3d} ({pct:5.1f}%)")

# # =====================================================================
# # 3. LENGTH & COMPLEXITY ANALYSIS
# # =====================================================================

# print("\n[3/4] Analyzing output length & complexity...")

# gold_lengths = []
# pred_lengths = []
# aspect_counts_gold = []
# aspect_counts_pred = []

# for item in test_predictions:
#     gold = item['gold_target'].strip()
#     pred = item['gen_output'].strip()
    
#     if gold:
#         gold_lengths.append(len(gold.split()))
#         aspect_counts_gold.append(gold.count(';') + 1)
    
#     if pred:
#         pred_lengths.append(len(pred.split()))
#         aspect_counts_pred.append(pred.count(';') + 1)

# print(f"\n✅ Output Length (in tokens):")
# print(f" Gold - Mean: {np.mean(gold_lengths):.1f}, Median: {np.median(gold_lengths):.1f}")
# print(f" Pred - Mean: {np.mean(pred_lengths):.1f}, Median: {np.median(pred_lengths):.1f}")
# print(f" Length Ratio (Pred/Gold): {np.mean(pred_lengths)/np.mean(gold_lengths):.2f}x")

# print(f"\n✅ Aspect Count per Sample:")
# print(f" Gold - Mean: {np.mean(aspect_counts_gold):.2f}")
# print(f" Pred - Mean: {np.mean(aspect_counts_pred):.2f}")

# # =====================================================================
# # 4. COMPREHENSIVE METRICS TABLE
# # =====================================================================

# print("\n" + "="*70)
# print("COMPREHENSIVE EVALUATION TABLE")
# print("="*70)

# comprehensive_results = pd.DataFrame({
#     'Category': [
#         'ABSA Metrics',
#         '',
#         'Text Quality',
#         '',
#         '',
#         '',
#         'Semantic',
#         '',
#         'Error Analysis',
#         '',
#         '',
#         '',
#         '',
#         'Structure',
#         ''
#     ],
#     'Metric': [
#         'Aspect+Sentiment F1',
#         'Aspect-Only F1',
#         'BLEU-4',
#         'ROUGE-1 F1',
#         'ROUGE-2 F1',
#         'ROUGE-L F1',
#         'BERTScore F1',
#         'Cosine Similarity',
#         'Perfect Matches',
#         'Parsing Errors',
#         'Empty Outputs',
#         'Sentiment Errors',
#         'Missing Fields',
#         'Perfect 7-Field %',
#         'Avg Fields/Entry'
#     ],
#     'Value': [
#         f"{tuple_f1:.4f}",  # ✅ FIX: use tuple_f1 from CELL 10
#         f"{aspect_f1:.4f}",  # ✅ FIX: use aspect_f1 from CELL 10
#         f"{avg_bleu:.4f}",
#         f"{avg_r1:.4f}",
#         f"{avg_r2:.4f}",
#         f"{avg_rl:.4f}",
#         f"{avg_bert_f1:.4f}",
#         f"{avg_cosine:.4f}",
#         f"{error_stats['perfect_match']}",
#         f"{error_stats['parsing_error']}",
#         f"{error_stats['empty_output']}",
#         f"{error_stats['wrong_sentiment']}",
#         f"{error_stats['missing_fields']}",
#         f"{format_adherence:.2%}",  # ✅ FIX: use format_adherence from CELL 10
#         f"{np.mean(field_counts):.2f}/7.0"  # ✅ FIX: use field_counts from CELL 10
#     ]
# })

# print(comprehensive_results.to_string(index=False))
# print("="*70)

# # Save comprehensive report
# final_report = {
#     "dataset": {
#         "total_samples": len(test_predictions),
#         "valid_predictions": len([x for x in test_predictions if x['gen_output'].strip()])
#     },
#     "absa_metrics": {
#         "aspect_sentiment_f1": float(tuple_f1),
#         "aspect_only_f1": float(aspect_f1),
#         "format_adherence": float(format_adherence)
#     },
#     "text_quality": {
#         "bleu4": float(avg_bleu),
#         "rouge1_f1": float(avg_r1),
#         "rouge2_f1": float(avg_r2),
#         "rougeL_f1": float(avg_rl)
#     },
#     "semantic_metrics": {
#         "bertscore_f1": float(avg_bert_f1),
#         "cosine_similarity": float(avg_cosine),
#         "span_match_ratio": float(avg_span_match)
#     },
#     "error_analysis": error_stats,
#     "length_analysis": {
#         "gold_mean_tokens": float(np.mean(gold_lengths)),
#         "pred_mean_tokens": float(np.mean(pred_lengths)),
#         "length_ratio": float(np.mean(pred_lengths) / np.mean(gold_lengths))
#     }
# }

# with open('final_evaluation_report.json', 'w') as f:
#     json.dump(final_report, f, indent=2)

# print(f"\n✓ Final report saved to final_evaluation_report.json")

# print("\n" + "="*70)
# print("✅ CELL 12 COMPLETE - EVALUATION FINISHED")
# print("="*70)


In [17]:
# # =======================================================================
# # CELL 13: DIAGNOSTIC - RANDOM SAMPLE INSPECTION
# # =======================================================================

# print("="*70)
# print("CELL 13: DETAILED SAMPLE INSPECTION")
# print("="*70)

# import random

# # Sample random predictions
# num_samples = 10
# sample_indices = random.sample(range(len(test_predictions)), min(num_samples, len(test_predictions)))

# print(f"\n📋 Showing {len(sample_indices)} random samples from test set\n")

# for i, idx in enumerate(sample_indices, 1):
#     item = test_predictions[idx]
    
#     print("="*70)
#     print(f"SAMPLE {i} (Index: {idx})")
#     print("="*70)
    
#     # Show input
#     print(f"\n📝 INPUT TEXT:")
#     print(f"{item['input'][:200]}..." if len(item['input']) > 200 else item['input'])
    
#     # Parse gold and pred
#     gold_parsed = parse_7field_robust(item['gold_target'])
#     pred_parsed = parse_7field_robust(item['gen_output'])
    
#     print(f"\n✅ GOLD TARGET ({len(gold_parsed)} aspects):")
#     print(f"{item['gold_target']}")
    
#     print(f"\n🤖 MODEL OUTPUT ({len(pred_parsed)} aspects):")
#     print(f"{item['gen_output']}")
    
#     # Detailed breakdown
#     print(f"\n📊 BREAKDOWN:")
#     print(f"{'':15} {'Gold':<40} {'Predicted':<40}")
#     print("-"*100)
    
#     max_len = max(len(gold_parsed), len(pred_parsed))
#     for j in range(max_len):
#         gold_aspect = gold_parsed[j]['aspect'] if j < len(gold_parsed) else "—"
#         gold_sent = gold_parsed[j]['sentiment'] if j < len(gold_parsed) else "—"
#         pred_aspect = pred_parsed[j]['aspect'] if j < len(pred_parsed) else "—"
#         pred_sent = pred_parsed[j]['sentiment'] if j < len(pred_parsed) else "—"
        
#         match = "✅" if (gold_aspect == pred_aspect and gold_sent == pred_sent) else "❌"
        
#         print(f"{match} Aspect {j+1}:   {gold_aspect:<40} {pred_aspect:<40}")
#         print(f"  Sentiment:  {gold_sent:<40} {pred_sent:<40}")
    
#     print()

# # =======================================================================
# # ADDITIONAL: Check for mode collapse
# # =======================================================================

# print("\n" + "="*70)
# print("🔍 MODE COLLAPSE CHECK")
# print("="*70)

# # Count unique predictions
# unique_predictions = {}
# for item in test_predictions:
#     pred = item['gen_output'].strip()
#     unique_predictions[pred] = unique_predictions.get(pred, 0) + 1

# print(f"\nTotal predictions: {len(test_predictions)}")
# print(f"Unique predictions: {len(unique_predictions)}")
# print(f"Repetition rate: {(1 - len(unique_predictions)/len(test_predictions))*100:.1f}%")

# # Show most common predictions
# print(f"\n📊 TOP 10 MOST COMMON PREDICTIONS:")
# sorted_preds = sorted(unique_predictions.items(), key=lambda x: x[1], reverse=True)

# for i, (pred, count) in enumerate(sorted_preds[:10], 1):
#     pct = count / len(test_predictions) * 100
#     print(f"\n{i}. Count: {count} ({pct:.1f}%)")
#     print(f"   Prediction: {pred[:150]}..." if len(pred) > 150 else f"   Prediction: {pred}")

# # =======================================================================
# # Check if model is always predicting same aspect
# # =======================================================================

# print("\n" + "="*70)
# print("🎯 ASPECT DIVERSITY CHECK")
# print("="*70)

# predicted_aspects = []
# for item in test_predictions:
#     parsed = parse_7field_robust(item['gen_output'])
#     for p in parsed:
#         predicted_aspects.append(p['aspect'])

# if predicted_aspects:
#     from collections import Counter
#     aspect_counts = Counter(predicted_aspects)
    
#     print(f"\nTotal predicted aspects: {len(predicted_aspects)}")
#     print(f"Unique aspects: {len(aspect_counts)}")
#     print(f"\nTop 10 most predicted aspects:")
#     for aspect, count in aspect_counts.most_common(10):
#         pct = count / len(predicted_aspects) * 100
#         print(f"  {aspect:<30} : {count:3d} ({pct:5.1f}%)")
# else:
#     print("\n⚠️  No aspects parsed from predictions!")

# print("\n" + "="*70)
# print("✅ CELL 13 COMPLETE - Check for mode collapse above")
# print("="*70)


In [18]:
# Add before training
print(f"Dataset sizes:")
print(f"  Train: {len(train_ds)}")
print(f"  Val: {len(val_ds)}")  
print(f"  Test: {len(test_ds)}")

# If training < 100 samples, you need more data!


Dataset sizes:
  Train: 2114
  Val: 452
  Test: 453


In [19]:
# CELL 14: TRAINING DATA INSPECTION
print("="*70)
print("CELL 14: TRAINING DATA DIAGNOSTIC")
print("="*70)

# Check dataset sizes
print(f"\n📊 Dataset Sizes:")
print(f"  Train: {len(train_ds)}")
print(f"  Val: {len(val_ds)}")
print(f"  Test: {len(test_ds)}")

# Sample training data
print(f"\n📋 Sample Training Data (first 3):")
for i in range(min(3, len(train_ds))):
    print(f"\n--- Training Sample {i+1} ---")
    print(f"Input: {train_ds[i]['input'][:200]}...")
    print(f"Target: {train_ds[i]['target']}")

# Check for "service_speed" in training data
service_speed_count = 0
for item in train_ds:
    if 'service_speed' in item['target'].lower():
        service_speed_count += 1

print(f"\n⚠️  'service_speed' appears in {service_speed_count}/{len(train_ds)} training samples")
print(f"   ({service_speed_count/len(train_ds)*100:.1f}%)")

print("\n" + "="*70)


CELL 14: TRAINING DATA DIAGNOSTIC

📊 Dataset Sizes:
  Train: 2114
  Val: 452
  Test: 453

📋 Sample Training Data (first 3):

--- Training Sample 1 ---
Input: Extract aspects, sentiments, and rationales. Output format: aspect | sentiment | span | rationale | is_implicit | has_slang | has_emoji
Separate multiple aspects with ' ; '.

Example 1 (Explicit, Mult...
Target: public_transport | negative | public transport fare | Explicit negative: fare 'hiked' without improvement. | FALSE | FALSE | FALSE

--- Training Sample 2 ---
Input: Extract aspects, sentiments, and rationales. Output format: aspect | sentiment | span | rationale | is_implicit | has_slang | has_emoji
Separate multiple aspects with ' ; '.

Example 1 (Explicit, Mult...
Target: service_speed | negative | ambulance wrong hospital destination | implicitly expresses negative sentiment about service_speed (not mentioned) via emergency transport misdirected, requiring inference that navigation procedures are catastrophically defici